# Training the models
In this notebook we will be training several different and increasingly more complex Deep Q-Learning models and saving their best result and run logs alongside. We will be experimenting with different types of greedy policies and improvements pon the deep learning model, such as: dueling networks and double deep q learning.

**NOTE ON RUNNING WITH TENSORFLOW** The infrequent updates actually make the GPU backend slower than CPU due to data transfer overhead. If using tensorflow as a backend for keras, it is best to use the CPU backend with optimizations.

In [1]:
import sys
import os

from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent, Sequential
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy, BoltzmannQPolicy
from rl.callbacks import FileLogger

# add local path
sys.path.append(os.path.join(os.getcwd(), "./src"))

from rl_model import TorcsKerasTrainer

/home/nikitautiu/envs/mirpr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
NB_OF_RUNS = 10  # the number of runs for each one of experiments
NB_STEPS = 100000  # the number of steps for each episode 

## Simple deep q-learning
We begin with defining the model for our most simple method. This is a simple deep q-learning model with an Epsilon Q-greedy policy. The epsilon is fixed at 5%. The deep model is a 3-layer 35-25-18 network with relu activations. 10 training of 10^5 steps each are performed,.

In [3]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=EpsGreedyQPolicy(0.05))
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

Run the simulation for the given number of times(10).

In [ ]:
NB_OF_RUNS = 10

for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/simple_dqn_best_{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/simple_dqn_{}.json'.format(run_nb))])

## Linearly annealed eps
Thi one is the same as the one before, but the epsilon starts large and slowly decreases over time. In theory this should increase exploration.

In [5]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.05,
                                  value_test=.0, nb_steps=40000)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=policy)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    
    return dqn

In [ ]:
for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/simple_dqn_linear_eps_best_{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/simple_dqn_linear_eps_{}.json'.format(run_nb))], min_step_save=40000)

## Boltzmann Exploration

In [7]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)
    policy = BoltzmannQPolicy()

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=policy)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    
    return dqn

In [ ]:
for run_nb in range(NB_OF_RUNS):
    yqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/simple_dqn_boltzmann_best_{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/simple_dqn_boltzmann_{}.json'.format(run_nb))])

## Dueling DQN
Uses two networks, one predicting the reward for a particular action, and one for the state. The results are then averaged.

In [9]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=EpsGreedyQPolicy(0.05), enable_dueling_network=True,
                   dueling_type='avg')
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

In [10]:
for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/dueling_dqn_best-{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/dueling_dqn_{}.json'.format(run_nb))])

Waiting for server on 3101............
Count Down : 5
Waiting for server on 3101............
Count Down : 4
Waiting for server on 3101............
Count Down : 3
Waiting for server on 3101............
Count Down : 2
Waiting for server on 3101............
Count Down : 1
Waiting for server on 3101............
Count Down : 0
Waiting for server on 3101............
Count Down : -1
relaunch torcs
Waiting for server on 3101............
Count Down : 4
Waiting for server on 3101............
Count Down : 3
Waiting for server on 3101............
Count Down : 2
Waiting for server on 3101............
Count Down : 1
Waiting for server on 3101............
Count Down : 0
Waiting for server on 3101............
Count Down : -1
relaunch torcs
Client connected on 3101..............
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_30 (Flatten)         (None, 29)                0         
_____________________________

 3021/10000 [========>.....................] - ETA: 2:01 - reward: 25.7573Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3066/10000 [========>.....................] - ETA: 2:01 - reward: 25.4345Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3130/10000 [========>.....................] - ETA: 2:01 - reward: 25.2888Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3246/10000 [========>.....................] - ETA: 1:58 - reward: 25.1575Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3313/10000 [========>.....................] - ETA: 1:57 - reward: 25.0526Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3380/10000 [=========>....................] - ETA: 1:57 - reward: 24.9151Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 6830/10000 [===================>..........] - ETA: 57s - reward: 23.9071Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6923/10000 [===================>..........] - ETA: 55s - reward: 23.9484Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7011/10000 [====================>.........] - ETA: 54s - reward: 23.9401Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7106/10000 [====================>.........] - ETA: 52s - reward: 24.1136Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7202/10000 [====================>.........] - ETA: 50s - reward: 24.2648Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7300/10000 [====================>.........] - ETA: 48s - reward: 24.2822Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 735

 1095/10000 [==>...........................] - ETA: 3:37 - reward: 20.2193Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1130/10000 [==>...........................] - ETA: 3:41 - reward: 20.1406Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1169/10000 [==>...........................] - ETA: 3:43 - reward: 19.6916Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1204/10000 [==>...........................] - ETA: 3:47 - reward: 19.7717Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1246/10000 [==>...........................] - ETA: 3:50 - reward: 20.4211Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1281/10000 [==>...........................] - ETA: 3:52 - reward: 20.4068Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 3816/10000 [==========>...................] - ETA: 2:46 - reward: 43.2029Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3869/10000 [==========>...................] - ETA: 2:45 - reward: 43.3560Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3924/10000 [==========>...................] - ETA: 2:44 - reward: 43.5340Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3996/10000 [==========>...................] - ETA: 2:42 - reward: 44.0125Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4048/10000 [===========>..................] - ETA: 2:41 - reward: 44.1879Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4097/10000 [===========>..................] - ETA: 2:40 - reward: 44.3076Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

...

 1939/10000 [====>.........................] - ETA: 3:42 - reward: 39.5476Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1974/10000 [====>.........................] - ETA: 3:43 - reward: 39.2719Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2031/10000 [=====>........................] - ETA: 3:42 - reward: 40.0289Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2104/10000 [=====>........................] - ETA: 3:39 - reward: 41.0436Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2274/10000 [=====>........................] - ETA: 3:28 - reward: 40.0408Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2339/10000 [======>.......................] - ETA: 3:27 - reward: 39.3949Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 4998/10000 [=============>................] - ETA: 2:13 - reward: 45.5949Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5027/10000 [==============>...............] - ETA: 2:13 - reward: 45.3607Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5073/10000 [==============>...............] - ETA: 2:12 - reward: 45.3863Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5111/10000 [==============>...............] - ETA: 2:12 - reward: 45.2848Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5220/10000 [==============>...............] - ETA: 2:08 - reward: 45.9049Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5348/10000 [===============>..............] - ETA: 2:04 - reward: 46.1656Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

10000/10000 [==============================] - 240s 24ms/step - reward: 40.4672
130 episodes - episode_reward: 3061.933 [-225.019, 8383.911] - loss: 13970.684 - mean_absolute_error: 1632.722 - mean_q: 1942.357

Interval 3 (20000 steps performed)
  144/10000 [..............................] - ETA: 1:51 - reward: 3.6615 Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
  277/10000 [..............................] - ETA: 2:26 - reward: 29.5622Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
  484/10000 [>.............................] - ETA: 2:26 - reward: 32.6908Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
  609/10000 [>.............................] - ETA: 2:31 - reward: 38.5268Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
  794/10000 [=>............................] - ETA: 2:28 - reward: 39.3351Waiting for 

 8239/10000 [=======================>......] - ETA: 24s - reward: 4.1598Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 8921/10000 [=========================>....] - ETA: 14s - reward: 4.7414Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9544/10000 [===========================>..] - ETA: 6s - reward: 5.3567Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
10000/10000 [==============================] - 137s 14ms/step - reward: 5.3455
19 episodes - episode_reward: 3048.260 [-158.636, 8826.764] - loss: 57060.910 - mean_absolute_error: 4761.176 - mean_q: 5352.364

Interval 5 (40000 steps performed)
   46/10000 [..............................] - ETA: 1:50 - reward: -18.0447Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
  548/10000 [>.............................] - ETA: 2:02 - reward: -0.7267Waiting for server o

 1189/10000 [==>...........................] - ETA: 2:04 - reward: 39.4225Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1340/10000 [===>..........................] - ETA: 2:05 - reward: 41.3609Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1449/10000 [===>..........................] - ETA: 2:08 - reward: 43.4812Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1596/10000 [===>..........................] - ETA: 2:09 - reward: 45.0298Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2151/10000 [=====>........................] - ETA: 1:57 - reward: 44.1004Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2626/10000 [======>.......................] - ETA: 1:49 - reward: 44.8630 ETA: 1:49 - rWaiting for server on 3101............
Count Down : 5
Client connected on 310

## Double DQN
This approach separates the model network from the q-value network. Basically a network predicts the actions(like a policy function) and the other one predicts the q value of the given action.

In [11]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=EpsGreedyQPolicy(0.05), enable_double_dqn=True)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

In [12]:
for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/double_dqn_best-{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/double_dqn_{}.json'.format(run_nb))])

Waiting for server on 3101............
Count Down : 5
Waiting for server on 3101............
Count Down : 4
Waiting for server on 3101............
Count Down : 3
Waiting for server on 3101............
Count Down : 2
Waiting for server on 3101............
Count Down : 1
Waiting for server on 3101............
Count Down : 0
Waiting for server on 3101............
Count Down : -1
relaunch torcs
Waiting for server on 3101............
Count Down : 4
Waiting for server on 3101............
Count Down : 3
Waiting for server on 3101............
Count Down : 2
Waiting for server on 3101............
Count Down : 1
Waiting for server on 3101............
Count Down : 0
Waiting for server on 3101............
Count Down : -1
relaunch torcs
Client connected on 3101..............
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_40 (Flatten)         (None, 29)                0         
_____________________________

  988/10000 [=>............................] - ETA: 3:40 - reward: 24.3629Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1053/10000 [==>...........................] - ETA: 3:39 - reward: 24.5068Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1136/10000 [==>...........................] - ETA: 3:37 - reward: 25.5629Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1219/10000 [==>...........................] - ETA: 3:34 - reward: 26.4387Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1263/10000 [==>...........................] - ETA: 3:37 - reward: 25.5556Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1356/10000 [===>..........................] - ETA: 3:33 - reward: 26.3000Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 4743/10000 [=============>................] - ETA: 2:07 - reward: 28.0906
Reward of 7978.682055044572 for episode 81 better than last -- SAVING

Client connected on 3101..............
 4888/10000 [=============>................] - ETA: 2:01 - reward: 28.8676Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4958/10000 [=============>................] - ETA: 2:00 - reward: 28.7117Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5074/10000 [==============>...............] - ETA: 1:57 - reward: 28.6530Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5144/10000 [==============>...............] - ETA: 1:56 - reward: 28.6097Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5282/10000 [==============>...............] - ETA: 1:52 - reward: 29.2810Waiting for server on 3101............
Count Down : 5
Client connected on

 9743/10000 [============================>.] - ETA: 5s - reward: 29.9564Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9814/10000 [============================>.] - ETA: 4s - reward: 29.7785Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9913/10000 [============================>.] - ETA: 1s - reward: 29.8105Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9960/10000 [============================>.] - ETA: 0s - reward: 29.7499Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
10000/10000 [==============================] - 225s 22ms/step - reward: 29.7854
115 episodes - episode_reward: 2577.560 [-819.595, 7978.682] - loss: 4366.406 - mean_absolute_error: 825.450 - mean_q: 923.831

Interval 3 (20000 steps performed)
   81/10000 [..............................] - ETA: 1:55 - reward: 66.9903Waiting for server on 3

 3454/10000 [=========>....................] - ETA: 2:33 - reward: 35.2806Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3521/10000 [=========>....................] - ETA: 2:32 - reward: 35.1295Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3592/10000 [=========>....................] - ETA: 2:31 - reward: 34.6184Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3651/10000 [=========>....................] - ETA: 2:30 - reward: 34.3983Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4332/10000 [===========>..................] - ETA: 2:04 - reward: 33.5839
Reward of 13689.639791634754 for episode 185 better than last -- SAVING

Client connected on 3101..............
 4461/10000 [============>.................] - ETA: 1:59 - reward: 33.9711Waiting for server on 3101............
Count Down : 5
Client connected 

...

 7153/10000 [====================>.........] - ETA: 1:04 - reward: 34.0512Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7379/10000 [=====================>........] - ETA: 59s - reward: 34.0610Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7418/10000 [=====================>........] - ETA: 58s - reward: 33.8747Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7468/10000 [=====================>........] - ETA: 57s - reward: 33.8399Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7640/10000 [=====================>........] - ETA: 53s - reward: 34.0569Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7788/10000 [======================>.......] - ETA: 49s - reward: 34.3323Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 78

 1294/10000 [==>...........................] - ETA: 3:32 - reward: 31.5195Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1339/10000 [===>..........................] - ETA: 3:34 - reward: 31.0728Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1392/10000 [===>..........................] - ETA: 3:34 - reward: 31.2791Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1449/10000 [===>..........................] - ETA: 3:34 - reward: 31.1684Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1480/10000 [===>..........................] - ETA: 3:37 - reward: 30.4274Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1541/10000 [===>..........................] - ETA: 3:36 - reward: 31.2296Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 4656/10000 [============>.................] - ETA: 2:13 - reward: 34.0221Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4683/10000 [=============>................] - ETA: 2:13 - reward: 33.7359Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4780/10000 [=============>................] - ETA: 2:10 - reward: 33.5805Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4903/10000 [=============>................] - ETA: 2:06 - reward: 34.2079Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4955/10000 [=============>................] - ETA: 2:05 - reward: 34.1313Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5263/10000 [==============>...............] - ETA: 1:55 - reward: 33.5416Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 2204/10000 [=====>........................] - ETA: 1:46 - reward: 33.2362Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2307/10000 [=====>........................] - ETA: 1:48 - reward: 33.8918Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2379/10000 [======>.......................] - ETA: 1:49 - reward: 34.4880Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2490/10000 [======>.......................] - ETA: 1:49 - reward: 36.0651Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2646/10000 [======>.......................] - ETA: 1:48 - reward: 35.7269Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 2824/10000 [=======>......................] - ETA: 1:47 - reward: 34.3499Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 4008/10000 [===========>..................] - ETA: 1:11 - reward: 43.4542Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4623/10000 [============>.................] - ETA: 1:05 - reward: 41.2089Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 8329/10000 [=======================>......] - ETA: 20s - reward: 47.2110Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 8492/10000 [========================>.....] - ETA: 18s - reward: 46.6769Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
10000/10000 [==============================] - 124s 12ms/step - reward: 47.7347
7 episodes - episode_reward: 118548.821 [3159.812, 589732.160] - loss: 87817.969 - mean_absolute_error: 9062.545 - mean_q: 9819.128

Interval 9 (80000 steps performed)
 5238/10000 [==============>...............] - ETA: 56s - reward: 54.9049Waiting for s